In [1]:
import os

In [2]:
%pwd

'c:\\Users\\nikhil\\OneDrive\\Desktop\\ML Projects\\Petrol-Price-Forecasting\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\nikhil\\OneDrive\\Desktop\\ML Projects\\Petrol-Price-Forecasting'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    p: int
    d: int
    q: int

In [10]:
from ppf.constants import *
from ppf.utils.common import create_directories, read_yaml

In [12]:

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:

        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])
        
        model_trainer_config =  ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            p = params.p,
            d = params.d,
            q = params.q
        )
        
        return model_trainer_config

In [71]:
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import pickle
from ppf.logging import logger

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
    
        dataset = pd.read_csv(self.config.data_path, index_col=False)  
        dataset.set_index('Date', inplace=True)
        dataset.index = pd.to_datetime(dataset.index)
        # print(dataset.index)

        model=ARIMA(dataset, order=(self.config.p, self.config.d, self.config.q))
        model_fit=model.fit()
        
        # save model
        pickle.dump(model, open(self.config.model_path, 'wb'))
        logger.info(f'Model is stored as {self.config.model_path}')

In [72]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config = model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-05-14 21:58:43,617: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-14 21:58:43,620: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-14 21:58:43,624: INFO: common: directory artifacts created]
[2024-05-14 21:58:43,627: INFO: common: directory artifacts/model_trainer created]
DatetimeIndex(['2003-06-01', '2003-07-01', '2003-08-01', '2003-09-01',
               '2003-10-01', '2003-11-01', '2003-12-01', '2004-01-01',
               '2004-02-01', '2004-03-01',
               ...
               '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01',
               '2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01',
               '2018-11-01', '2018-12-01'],
              dtype='datetime64[ns]', name='Date', length=187, freq=None)


c:\Users\nikhil\.conda\envs\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\nikhil\.conda\envs\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\nikhil\.conda\envs\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


[2024-05-14 21:58:44,629: INFO: 3592429050: Model is stored as artifacts/model_trainer/model.pkl]


c:\Users\nikhil\.conda\envs\venv\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
